###step:load the workspace configuration

In [3]:
from azureml.core import Workspace,Dataset,Experiment

subscription_id = 'dfb14526-a13a-4985-8434-3a0c79be2a0f'
resource_group = 'mlops-aug-batch'
workspace_name = 'intellipat-mlops'

workspace = Workspace(subscription_id, resource_group, workspace_name)

In [4]:
experment = Experiment(workspace= workspace,name="iris")
dataset = Dataset.get_by_name(workspace=workspace,name="Iris")

df=dataset.to_pandas_dataframe()
df.head(4)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa


In [5]:
X =df[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']]
Y =df['Species']

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size= 0.10,shuffle=True)


****###create a DT classifier

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


#step1:initialise the model class
clf =DecisionTreeClassifier()
#step2:train the model with training data
clf.fit(X_train,y_train)

#step3: evaluated the model with testing data 
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test,y_pred)


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X =df[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']]
Y =df['Species']

run=experment.start_logging()


X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size= 0.10,shuffle=True)


#step1:initialise the model class
clf =DecisionTreeClassifier(criterion="gini",max_depth=4)
#step2:train the model with training data
clf.fit(X_train,y_train)

#step3: evaluated the model with testing data 
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test,y_pred)

run.log("accuracy",accuracy*100)

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X =df[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']]
Y =df['Species']

run=experment.start_logging()


X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size= 0.10,shuffle=True)


#step1:initialise the model class
clf =DecisionTreeClassifier()
#step2:train the model with training data
clf.fit(X_train,y_train)

#step3: evaluated the model with testing data 
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test,y_pred)

run.log("accuracy",accuracy*100)

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import joblib

X = df[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']]
Y = df['Species']

run = experment.start_logging()
test_size=0.40

run.log("train_test_split",test_size)

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size= test_size, shuffle=True)


#step1: initialise the model class
clf = DecisionTreeClassifier(criterion="gini", max_depth=4)
#step:2 train the model with training data
clf.fit(X_train, y_train)

#step-3 evaluate the model with testing data
y_pred = clf.predict(X_test)
accurcay = accuracy_score(y_test,y_pred)

run.log("accuracy",accurcay*100 )

model_name =  "iris_decision_tree.pkl"
filepath = "outputs/"+model_name

joblib.dump(value= clf,filename=filepath)
run.upload_file(name=model_name, path_or_stream=filepath)
run.complete()

### step 4: Register the model


In [14]:
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration
import sklearn

In [15]:
model = Model.register(
    workspace=workspace,
    model_name = "iris",
    model_path = filepath,
    model_framework= Model.Framework.SCIKITLEARN,
    model_framework_version= sklearn.__version__,
    description= "This is a sample DT classifier",
    resource_configuration= ResourceConfiguration(cpu=1, memory_in_gb= 0.5)
)

print("Name:", model.name)
print("version:", model.version)

Registering model iris
Name: iris
version: 1


In [10]:
!pip show scikit-learn

Name: scikit-learn
Version: 0.22.1
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: None
Author-email: None
License: new BSD
Location: /anaconda/envs/azureml_py38/lib/python3.8/site-packages
Requires: scipy, numpy, joblib
Required-by: sklearn-pandas, skl2onnx, shap, seqeval, responsibleai, raiwidgets, raiutils, pmdarima, ml-wrappers, lightgbm, interpret-community, fairlearn, erroranalysis, econml, dice-ml, azureml-training-tabular, azureml-train-automl-runtime, azureml-datadrift, azureml-automl-runtime, autokeras
